# Domácí úkol:
Pomocí vlastních tříd a dědičnosti vytvořte "soubor tříd" = nástroj pro derivaci funkcí.

## Jak to má fungovat:

- derivace je jednoduše algoritmizovatelná, neboť se řídí prakticky pouze třemi pravidly (podílu se dá vyhnout pomocí násobení funkcí $\frac{1}{f(x)}$):
    - derivace součtu je součet derivací
    - pravidlo pro derivaci součinu
    - derivace funkce je derivovaná funkce násobená derivovaným argumentem

- tedy stačí implementovat tyto tři základní pravidla a pro každou elementární funkci (sin, cos, exp, ln, x^n, ...) vědět jak vypadá její derivace
- celý tento proces je možné reprezentovat pomocí tříd a dědičnosti
- trik je v tom, že se výrazy automaticky skládají do stromu (pro nás v podstatě skrytého), a tedy každý výraz je na venek pouze buď sčítání, násobení, nebo elementární funkce (a o jejich argumentech platí totéž)

## Podrobnější zadání:
Konkrétněji:
- Vytvořte třídu **Operator**
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor (zde stačí `pass`)
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
- Vytvořte třídu **BinaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `levy_argument` - levý operand
        - `pravy_argument` - pravý operand
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argumenty
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru
- Vytvořte třídu **UnaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `argument` - argument funkce 
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argument
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru

Pro počítání derivací použijte standardní pravidla:
- derivace součtu je součet derivací
- pravidlo pro derivaci součinu
- derivace funkce je derivovaná funkce násobená derivovaným argumentem

Pro **BinaryOperator** a **UnaryOperator** implementujte také metody implementující standardní operátory:
- `__add__()` - implementuje operátor `+`
- `__sub__()` - implementuje operátor `-`
- `__mul__()` - implementuje operátor `*`

Pro **BinaryOperator** implementujte dvě dceřiné třídy:
- **Addition** - implementuje operátor `+`
- **Multiplication** - implementuje operátor `*`

Pro **UnaryOperator** implementujte tyto dceřiné třídy:
- **exp** - implementuje funkci `exp`
- **sin** - implementuje funkci `sin`
- **cos** - implementuje funkci `cos`
- **ln** - implementuje funkci `ln`
- **frc** - implementuje funkci `1/x`
- **identity** - implementuje identitu `x`
- **cst** - implementuje konstantu `c`
- **mocnina** - implementuje mocninu `x^n`

U všech funckí implementujte adekvátně metodu `diff()` a `__str__()`, tak aby se funkce vypisovaly v přehledné podobě.

Ozkoušejte na následujících funkcích:
- $sin(x + cos(x*x)) + 1$
- $x^2 + 2x + 1 + 2x^2$
- $x^2 + 2x + exp(x)$

In [70]:
# kostra k doplnění, místa označená TODO doplňte

class Operator:
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    
    def __repr__(self):
        return self.__str__()
    
    def diff(self):
        pass

    def simplify(self):
        return self
    
    def __add__(self, other):
        if type(other) == int or type(other) == float:
            other = cst(other)
        return Addition(self, other)
    
    def __radd__(self, other):
        if type(other) == int or type(other) == float:
            other = cst(other)
        return Addition(other, self)
    
    def __sub__(self, other):
        if type(other) == int or type(other) == float:
            other = cst(other)
        return Addition(self, Multiplication(cst(-1), other))
    
    def __rsub__(self, other):
        if type(other) == int or type(other) == float:
            other = cst(other)
        return Addition(Multiplication(cst(-1), other), self)
    
    def __mul__(self, other):
        if type(other) == int or type(other) == float:
            other = cst(other)
        return Multiplication(self, other)
    
    def __rmul__(self, other):
        if type(other) == int or type(other) == float:
            other = cst(other)
        return Multiplication(other, self)
    
class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument
        
class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument
    
    def __str__(self):
        return type(self).__name__ + "(" + str(self.argument) + ")"
        
class Addition(BinaryOperator):
    def __str__(self):
        return str(f"{self.levy_argument} + {self.pravy_argument}")
    def diff(self):
        return Addition(self.levy_argument.diff(), self.pravy_argument.diff()).simplify()
    def simplify(self):
        nula = str(cst(0))
        self.levy_argument = self.levy_argument.simplify()
        self.pravy_argument = self.pravy_argument.simplify()
        if str(self.levy_argument) == nula:
            return self.pravy_argument
        elif str(self.pravy_argument) == nula:
            return self.levy_argument
        elif type(self.levy_argument) == cst and type(self.pravy_argument) == cst:
            return cst(int(str(self.levy_argument)) + int(str(self.pravy_argument)))
        else:
            return self

class Multiplication(BinaryOperator):
    def __str__(self):
        return str(f"({self.levy_argument})*({self.pravy_argument})")
    def diff(self):
        return Addition(Multiplication(self.levy_argument.diff(), self.pravy_argument), Multiplication(self.levy_argument, self.pravy_argument.diff())).simplify()
    def simplify(self):
        nula = str(cst(0))
        jedna = str(cst(1))
        self.levy_argument = self.levy_argument.simplify()
        self.pravy_argument = self.pravy_argument.simplify()
        if str(self.levy_argument) == jedna:
            return self.pravy_argument
        elif str(self.pravy_argument) == jedna:
            return self.levy_argument
        elif str(self.levy_argument) == nula or str(self.pravy_argument) == nula:
            return cst(0)
        elif str(self.levy_argument) == str(self.pravy_argument):
            return mocnina(self.levy_argument, cst(2))
        elif type(self.levy_argument) == cst and type(self.pravy_argument) == cst:
            return cst(int(str(self.levy_argument)) * int(str(self.pravy_argument)))
        else:
            return self
    
class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return str(self.value)
    def diff(self):
        return cst(0)
    
class identity(UnaryOperator):
    def __init__(self):
        pass
    def __str__(self):
        return "x"
    def diff(self):
        return cst(1)
    
class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent
    def __str__(self):
        return str(f"({self.argument})^{self.exponent}")
    def diff(self):
        return Multiplication(Multiplication(cst(self.exponent), mocnina(self.argument, self.exponent - 1)), self.argument.diff()).simplify()
    def simplify(self):
        nula = str(cst(0))
        jedna = str(cst(1))
        self.argument = self.argument.simplify()
        if str(self.exponent) == nula:
            return cst(1)
        elif str(self.exponent) == jedna:
            return self.argument
        elif str(self.argument) == nula:
            return cst(0)
        else:
            return self
    
class sin(UnaryOperator):
    def diff(self):
        return Multiplication(cos(self.argument), self.argument.diff()).simplify()

class cos(UnaryOperator):
    def diff(self):
        return Multiplication(cst(-1), Multiplication(cos(self.argument), self.argument.diff())).simplify()

class exp(UnaryOperator):
    def diff(self):
        return Multiplication(self, self.argument.diff()).simplify()
    
class ln(UnaryOperator):
    def diff(self):
        return Multiplication(mocnina(self.argument, cst(-1)), self.argument.diff()).simplify()

class frc(UnaryOperator):
    def __str__(self):
        return str(f"1/({self.argument})")
    def diff(self):
        return Multiplication(Multiplication(cst(-1), frc(mocnina(self.argument, 2))), self.argument.diff()).simplify()

In [49]:
# takto by se to mělo používat
x = identity()
# sin(x + cos(x*x)) + 1
f1 = sin(x + cos(x*x)) + cst(1)
print(f1)
print(f1.diff())

sin(x + cos((x)*(x))) + 1
(cos(x + cos((x)*(x))))*(1 + (-1)*((cos((x)*(x)))*(x + x)))


můj výstup byl:

`sin(x + cos((x)*(x))) + 1`

`(cos(x + cos((x)*(x))))*(1 + ((-1)*(sin((x)*(x))))*((1)*(x) + (x)*(1))) + 0`

In [74]:
x = identity()
# x ^ 2 + 2x + 1 + 2x ^ 2
f2 = mocnina(x,2) + cst(2)*x + cst(1) + cst(2)*mocnina(x,2)
print(f2)
print(f2.diff())


(x)^2 + (2)*(x) + 1 + (2)*((x)^2)
(2)*(x) + 2 + (2)*((2)*(x))


můj výstup byl:

`(x)^2 + (2)*(x) + 1 + (2)*((x)^2)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + 0 + (0)*((x)^2) + (2)*(((2)*((x)^1))*(1))`

In [51]:
x = identity()
# x ^ 2 + 2x + exp(x)
f3 = mocnina(x,2) + cst(2)*x + exp(x)
print(f3)
print(f3.diff())


(x)^2 + (2)*(x) + exp(x)
(2)*(x) + 2 + exp(x)


můj výstup byl:

`(x)^2 + (2)*(x) + exp(x)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + (exp(x))*(1)`


In [52]:
# ln(exp(x))
f4 = ln(exp(x))
print(f4)
print(f4.diff())

ln(exp(x))
((exp(x))^-1)*(exp(x))


In [80]:
# (x^2 + 1) / (x^2 - 1)
f5 = (mocnina(x, 2) + cst(1)) * (mocnina(x, 2) + cst(-1))
print(f5)
print(f5.diff())

((x)^2 + 1)*((x)^2 + -1)
((2)*(x))*((x)^2 + -1) + ((x)^2 + 1)*((2)*(x))


# Bonusový úkol:
Doplňte ke všem třídám metodu `simplify()`, která provede zjednodušení výrazu. Například:
- součet s nulou vrátí druhý operand
- součin s nulou vrátí nulu
- mocnina s exponentem 0 vrátí 1
- mocnina s exponentem 1 vrátí první argument
- součín dvou stejných argumentů vrátí mocninu s exponentem 2
- součin konstant vrátí konstantu se součinem
- ...

Doplňte metody `__add__, __radd__, __mul__, __rmul__, ...` tak aby podporovaly operace s čísly (např. `2 + x` nebo `x * 3`), a zjednodušovaly zadávání výrazů. 

In [77]:
x = identity()
f6 = 2 + 6 * x * 3 + 1
print(f6)
print(f6.diff())

2 + ((6)*(x))*(3) + 1
18
